
# Pandas Fundamentals & Data Access (Finance-ready)

**Study path:** This notebook compresses everything you need for a FinTech / Quant dev master:
- DataFrame essentials (indexing, selecting, adding/removing, `.loc` / `.iloc`, multi-index)
- Conditional selection & multiple conditions
- Missing data (`dropna`, `fillna`), thresholds & axis
- GroupBy (aggregate, describe, count/min/max), wide-to-narrow intuition
- Concatenate / Merge / Join (axis, keys, inner/outer/left/right)
- Handy operations: `unique`, `nunique`, `value_counts`, `apply` (with lambdas), sorting, `isnull`
- Pivot tables
- I/O: CSV, Excel, HTML (via `StringIO`), SQL (SQLite) with **SQLAlchemy 2.x** pattern (fallback to `sqlite3`)
- Exercises with solutions (E1–E3)
- DataReader (example), Quandl API (free endpoints)

_Tip:_ Run cell-by-cell. Everything is self-contained and robust as of 2025.



## Table of Contents
1. [Imports & Setup](#imports)
2. [DataFrame Basics](#df-basics)
3. [Indexing & Selection (`[]`, `.loc`, `.iloc`)](#indexing)
4. [Add/Drop Columns & Rows; `axis`; `shape`](#add-drop)
5. [Conditional Selection & Multiple Conditions](#conds)
6. [Missing Data (`dropna`, `fillna`)](#missing)
7. [GroupBy & Aggregations](#groupby)
8. [Concatenate / Merge / Join](#merge)
9. [Operations: unique, value_counts, apply, sort, isnull](#ops)
10. [Pivot Tables](#pivot)
11. [I/O: CSV, Excel, HTML, SQL (SQLAlchemy 2.x)](#io)
12. [Exercises (E1–E3) + Solutions](#ex)
13. [Pandas DataReader (example) & Quandl](#external)


<a id='imports'></a>

## 1) Imports & Setup

In [1]:

import pandas as pd
import numpy as np
import io

# Optional SQLAlchemy
try:
    from sqlalchemy import create_engine, text
    SQLALCHEMY_OK = True
except Exception:
    SQLALCHEMY_OK = False
    print("SQLAlchemy not available -> SQL cells will use sqlite3 fallback.")

pd.set_option("display.precision", 4)


SQLAlchemy not available -> SQL cells will use sqlite3 fallback.


<a id='df-basics'></a>

## 2) DataFrame Basics

In [2]:

np.random.seed(101)
df = pd.DataFrame(np.random.randn(5,4), index=list("ABCDE"), columns=list("WXYZ"))
df


,W,X,Y,Z
A,2.7068,0.6281,0.9080,0.5038
B,0.6511,-0.3193,-0.8481,0.6060
C,-2.0182,0.7401,0.5288,-0.5890
D,0.1887,-0.7589,-0.9332,0.9551
E,0.1908,1.9788,2.6060,0.6835


<a id='indexing'></a>

## 3) Indexing & Selection

In [3]:

# Column(s)
df['W']              # one column -> Series
df[['W','Z']]        # list of columns -> DataFrame


,W,Z
A,2.7068,0.5038
B,0.6511,0.6060
C,-2.0182,-0.5890
D,0.1887,0.9551
E,0.1908,0.6835


In [4]:

# Rows with .loc (label) and .iloc (position)
df.loc['C']
df.iloc[2]


W   -2.0182
X    0.7401
Y    0.5288
Z   -0.5890
Name: C, dtype: float64

In [5]:

# Subsets row x column
df.loc['B','Y']                 # single value
df.loc[['A','B'], ['W','Y']]    # 2x2 subset


,W,Y
A,2.7068,0.9080
B,0.6511,-0.8481


<a id='add-drop'></a>

## 4) Add / Drop & `axis` / `shape`

In [6]:

# Add a column from others
df['NEW'] = df['W'] + df['Y']
df.shape, df.head(2)


((5, 5),
         W       X       Y       Z     NEW
 A  2.7068  0.6281  0.9080  0.5038  3.6148
 B  0.6511 -0.3193 -0.8481  0.6060 -0.1970)

In [7]:

# Drop column (axis=1); not in-place by default
df2 = df.drop('NEW', axis=1)
df2.columns, df.columns


(Index(['W', 'X', 'Y', 'Z'], dtype='object'),
 Index(['W', 'X', 'Y', 'Z', 'NEW'], dtype='object'))

In [8]:

# In-place drop
df.drop('NEW', axis=1, inplace=True)
'NEW' in df.columns


False

In [9]:

# Drop row (axis=0)
df.drop('E', axis=0)


,W,X,Y,Z
A,2.7068,0.6281,0.9080,0.5038
B,0.6511,-0.3193,-0.8481,0.6060
C,-2.0182,0.7401,0.5288,-0.5890
D,0.1887,-0.7589,-0.9332,0.9551


<a id='conds'></a>

## 5) Conditional Selection & Multiple Conditions

In [10]:

# Entire DF boolean mask
mask = df > 0
df[mask].head()


,W,X,Y,Z
A,2.7068,0.6281,0.9080,0.5038
B,0.6511,NaN,NaN,0.6060
C,NaN,0.7401,0.5288,NaN
D,0.1887,NaN,NaN,0.9551
E,0.1908,1.9788,2.6060,0.6835


In [11]:

# Filter rows by a column condition
df[df['W'] > 0]


,W,X,Y,Z
A,2.7068,0.6281,0.9080,0.5038
B,0.6511,-0.3193,-0.8481,0.6060
D,0.1887,-0.7589,-0.9332,0.9551
E,0.1908,1.9788,2.6060,0.6835


In [12]:

# Multiple conditions: use & (and) and | (or), each condition in parentheses
df[(df['W'] > 0) & (df['Y'] > 0)]


,W,X,Y,Z
A,2.7068,0.6281,0.908,0.5038
E,0.1908,1.9788,2.606,0.6835


<a id='missing'></a>

## 6) Missing Data (`dropna`, `fillna`)

In [13]:

d = {'A':[1,2,np.nan], 'B':[5,np.nan,np.nan], 'C':[1,2,3]}
mdf = pd.DataFrame(d)
display(mdf)
display(mdf.dropna())                 # drop rows with any NaN
display(mdf.dropna(axis=1))           # drop columns with any NaN
display(mdf.dropna(thresh=2))         # keep rows with >=2 non-NaN


,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2
2,NaN,NaN,3


,A,B,C
0,1.0,5.0,1


,C
0,1
1,2
2,3


,A,B,C
0,1.0,5.0,1
1,2.0,NaN,2


In [14]:

# Fill with value or statistic
mdf['A'] = mdf['A'].fillna(mdf['A'].mean())
mdf.fillna('FILL').head()


,A,B,C
0,1.0,5.0,1
1,2.0,FILL,2
2,1.5,FILL,3


<a id='groupby'></a>

## 7) GroupBy & Aggregations

In [15]:

data = {'Company':['GOOG','GOOG','MSFT','MSFT','FB','FB'],
        'Person':['Sam','Charlie','Amy','Vanessa','Carl','Sarah'],
        'Sales':[200,120,340,124,243,350]}
gdf = pd.DataFrame(data)
grp = gdf.groupby('Company')
display(grp.mean(numeric_only=True))
display(grp.sum(numeric_only=True))
display(grp.describe())


,Sales
Company,
FB,296.5
GOOG,160.0
MSFT,232.0


,Sales
Company,
FB,593
GOOG,320
MSFT,464


Sales                                                      
        count   mean       std    min     25%    50%     75%    max
Company                                                            
FB        2.0  296.5   75.6604  243.0  269.75  296.5  323.25  350.0
GOOG      2.0  160.0   56.5685  120.0  140.00  160.0  180.00  200.0
MSFT      2.0  232.0  152.7351  124.0  178.00  232.0  286.00  340.0

<a id='merge'></a>

## 8) Concatenate / Merge / Join

In [16]:

f1 = pd.DataFrame({'A':['A0','A1','A2','A3'],
                   'B':['B0','B1','B2','B3'],
                   'C':['C0','C1','C2','C3'],
                   'D':['D0','D1','D2','D3']},
                  index=[0,1,2,3])
f2 = f1.copy(); f2.index = [4,5,6,7]
f3 = f1.copy(); f3.index = [8,9,10,11]
pd.concat([f1,f2,f3], axis=0)


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A0,B0,C0,D0
5,A1,B1,C1,D1
6,A2,B2,C2,D2
7,A3,B3,C3,D3
8,A0,B0,C0,D0
9,A1,B1,C1,D1


In [17]:

left  = pd.DataFrame({'key':['K0','K1','K2','K3'],'A':['A0','A1','A2','A3'],'B':['B0','B1','B2','B3']})
right = pd.DataFrame({'key':['K0','K1','K2','K3'],'C':['C0','C1','C2','C3'],'D':['D0','D1','D2','D3']})
pd.merge(left, right, on='key', how='inner')


,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


In [18]:

# Join by index
l = left.set_index('key')
r = right.set_index('key')
l.join(r, how='inner')


,A,B,C,D
key,,,,
K0,A0,B0,C0,D0
K1,A1,B1,C1,D1
K2,A2,B2,C2,D2
K3,A3,B3,C3,D3


<a id='ops'></a>

## 9) Operations: unique / value_counts / apply / sort / isnull

In [19]:

ops = pd.DataFrame({'col1':[1,2,3,4], 'col2':[444,555,444,666], 'col3':['abc','def','ghi','xyz']})
ops['col2'].unique(), ops['col2'].nunique(), ops['col2'].value_counts()


(array([444, 555, 666]),
 3,
 col2
 444    2
 555    1
 666    1
 Name: count, dtype: int64)

In [20]:

# apply with function / lambda
def times2(x): return x*2
ops['col1'].apply(times2), ops['col3'].apply(len)


(0    2
 1    4
 2    6
 3    8
 Name: col1, dtype: int64,
 0    3
 1    3
 2    3
 3    3
 Name: col3, dtype: int64)

In [21]:

# sort, isnull
ops.sort_values('col2')
ops.isnull().head()


,col1,col2,col3
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False


<a id='pivot'></a>

## 10) Pivot Tables

In [22]:

pt = pd.DataFrame({'A':['foo','foo','bar','bar','foo','bar','foo','bar'],
                   'B':['one','one','two','three','two','two','one','three'],
                   'C':['x','y','x','y','x','y','x','x'],
                   'D':np.random.randn(8)})
pt.pivot_table(values='D', index=['A','B'], columns='C')


C               x       y
A   B                    
bar three  0.1845 -1.1591
    two   -1.7061  0.3905
foo one    0.2348  1.6937
    two   -0.1348     NaN

<a id='io'></a>

## 11) I/O: CSV, Excel, HTML (`StringIO`), SQL (SQLAlchemy 2.x)

In [23]:

# CSV
demo = pd.DataFrame({"A":[1,2,3], "B":[10,20,30]})
demo.to_csv("demo.csv", index=False)
pd.read_csv("demo.csv").head()


,A,B
0,1,10
1,2,20
2,3,30


In [24]:

# Excel
demo.to_excel("demo.xlsx", sheet_name="Sheet1", index=False)
pd.read_excel("demo.xlsx", sheet_name="Sheet1").head()


ModuleNotFoundError: No module named 'openpyxl'

In [ ]:

# HTML from literal string -> use StringIO (future-proof)
html = '''
<table>
  <tr><th>name</th><th>value</th></tr>
  <tr><td>alpha</td><td>1</td></tr>
  <tr><td>beta</td><td>2</td></tr>
</table>
'''
pd.read_html(io.StringIO(html))[0]


In [ ]:

# SQL: SQLite in-memory; SQLAlchemy 2.x pattern (fallback to sqlite3)
import sqlite3

if SQLALCHEMY_OK:
    engine = create_engine("sqlite+pysqlite:///:memory:", echo=False, future=True)
    demo.to_sql("my_table", engine, index=False)
    pd.read_sql("SELECT * FROM my_table", con=engine)
else:
    con = sqlite3.connect(":memory:")
    demo.to_sql("my_table", con, index=False)
    pd.read_sql("SELECT * FROM my_table", con)


<a id='ex'></a>

## 12) Exercises (with solutions)

**E1.** Write `demo` to `result.csv` and read it back as `df_loaded`. Verify equality.

In [ ]:

demo.to_csv("result.csv", index=False)
df_loaded = pd.read_csv("result.csv")
print("Equal values:", df_loaded.equals(demo))
df_loaded


**E2.** Append a second sheet (`Sheet2`) to `demo.xlsx` with a new DataFrame and read it back.

In [ ]:

other = pd.DataFrame({"C":[100,200], "D":[300,400]})
try:
    with pd.ExcelWriter("demo.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
        other.to_excel(writer, sheet_name="Sheet2", index=False)
except Exception:
    # Fall back: recreate file with both sheets
    with pd.ExcelWriter("demo.xlsx", engine="xlsxwriter") as writer:
        demo.to_excel(writer, sheet_name="Sheet1", index=False)
        other.to_excel(writer, sheet_name="Sheet2", index=False)
pd.read_excel("demo.xlsx", sheet_name="Sheet2")


**E3.** Create a SQL table `names`, insert two rows, and `SELECT` rows whose `first` starts with `'A'`.

In [ ]:

if SQLALCHEMY_OK:
    engine = create_engine("sqlite+pysqlite:///:memory:", echo=False, future=True)
    with engine.begin() as conn:
        conn.execute(text("CREATE TABLE names (first TEXT, last TEXT)"))
        conn.execute(text("INSERT INTO names VALUES (:f, :l)"), [{"f":"Ada","l":"Lovelace"},{"f":"Alan","l":"Turing"}])
    pd.read_sql("SELECT * FROM names WHERE first LIKE 'A%'", con=engine)
else:
    con = sqlite3.connect(":memory:")
    cur = con.cursor()
    cur.execute("CREATE TABLE names (first TEXT, last TEXT)")
    cur.executemany("INSERT INTO names VALUES (?,?)", [("Ada","Lovelace"),("Alan","Turing")])
    con.commit()
    pd.read_sql("SELECT * FROM names WHERE first LIKE 'A%'", con)


<a id='external'></a>

## 13) Pandas DataReader (example) & Quandl


**Note:** Internet access may be disabled in your environment. Treat the following as reference templates.

### Pandas DataReader (Google/other sources may change; adjust source accordingly)
```python
import pandas_datareader.data as web
from datetime import datetime

start = datetime(2015,1,1)
end   = datetime(2017,1,1)
fb = web.DataReader('FB', 'stooq', start, end)  # example: 'stooq' often works
fb.head()
```

### Quandl (free usage up to a daily limit)
```python
import quandl
# Simple time series (e.g., WTI crude from DOE)
oil = quandl.get('EIA/PET_RWTC_D')
oil.plot()

# Wiki end-of-day (legacy): 'WIKI/AAPL' (dataset availability can change)
aapl = quandl.get('WIKI/AAPL')
aapl[['Adj. Close','Adj. Volume']].tail()
```
